In [54]:
import httpx
from api.src.schemas.authorities.authority import Uri, Variant, AdminMetadata, Element
from typing import Optional
from pydantic import BaseModel
from pysolr import Solr
from pyfuseki import FusekiUpdate
from api.src.function.authorities.generateID import GenerateId
from api.src.schemas.authorities.subject import UriDelete


ModuleNotFoundError: No module named 'src'

In [7]:
id = GenerateId()
id

'bkau-2'

In [77]:
au_update = FusekiUpdate('http://localhost:3030', 'authorities')

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = au_update.run_sparql(d)
response.convert()

solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)
solr.delete(q="*:*",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">42</int>\n</lst>\n</response>\n'

In [82]:
authority = 'http://id.loc.gov/authorities/subjects/sh85102839'
url = f'http://localhost:8000/authorities/import/loc/subject?uri={authority}'
r = httpx.get(url, timeout=20)
r.status_code

200

In [83]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
response = httpx.post('http://localhost:8000/authorities/subject/', headers=headers, json=r.json())
response.json()  

{'id': 'bkau-3',
 'jena': 'Update succeeded',
 'solr': '{\n  "responseHeader":{\n    "status":0,\n    "QTime":113}}\n'}

In [55]:
class Subject(BaseModel):
    type: str 
    adminMetadata: AdminMetadata 
    elementList: list[Element]
    note: Optional[str]
    hasReciprocalAuthority: Optional[list[Uri]]
    hasReciprocalExternalAuthority: Optional[list[Uri]]
    hasBroaderAuthority: Optional[list[Uri]]
    hasBroaderExternalAuthority: Optional[list[Uri]]
    hasNarrowerAuthority: Optional[list[Uri]]
    hasNarrowerExternalAuthority: Optional[list[Uri]]
    hasCloseExternalAuthority: Optional[list[Uri]]
    hasExactExternalAuthority: Optional[list[Uri]]
    hasVariant: Optional[list[Variant]]
    subjectOf: Optional[list[Uri]]
    isMemberOfMADSCollection: str

class UriDelete(BaseModel):
    authority: str
    uri: str
    type: str

In [10]:
request = Subject(**r.json())

In [69]:
r = solr.search(q='id:bkau-2', **{'fl': '*,[child]'})
[doc] = r.docs
doc

{'id': 'bkau-2',
 'type': ['Topic'],
 'creationDate': ['2023-05-22T00:00:00Z'],
 'label': ['Plantas'],
 'isMemberOfMADSCollection': ['http://bibliokeia.com/authorities/Topic/'],
 'variant': ['Flora',
  'Reino vegetal',
  'Plantae',
  'Plantas vasculares',
  'Reino vegetal',
  'Vegetação',
  'Animais selvagens'],
 '_version_': 1766616444907290624,
 'hasReciprocalAuthority': {'id': 'bkau-2/hasReciprocalAuthority#bkau-1',
  'uri': ['https://bibliokeia.com/authorities/Topic/bkau-1'],
  'label': ['Botânica'],
  'lang': ['por'],
  'base': ['bk'],
  '_version_': 1766616444907290624},
 'hasBroaderExternalAuthority': {'id': 'bkau-2/hasBroaderExternalAuthority#sh2003007697',
  'uri': ['http://id.loc.gov/authorities/subjects/sh2003007697'],
  'label': ['Organisms'],
  'lang': ['en'],
  'base': ['loc'],
  '_version_': 1766616444907290624},
 'hasNarrowerExternalAuthority': [{'id': 'bkau-2/hasNarrowerExternalAuthority#sh98000620',
   'uri': ['http://id.loc.gov/authorities/subjects/sh98000620'],
   '

In [76]:
uri = 'https://bibliokeia.com/authorities/Topic/bkau-1'
def UpdateUri(metadado, typeUri, authority ):

    if doc.get(metadado):
        if type(doc.get(metadado)) == list:
            pass
        else:
            [delUri] = doc.get(metadado)['uri']
            request = {'authority': authority, 
                            'uri': delUri, 
                            'type': typeUri }
            request = UriDelete(**request)
            resposneUpdate = UpdateDeleteUri(request, typeUri)
            print(request)
UpdateUri(
    'hasReciprocalAuthority', 
    'hasReciprocalAuthority', 
    'https://bibliokeia.com/authorities/Topic/bkau-1' )

authority='https://bibliokeia.com/authorities/Topic/bkau-1' uri='https://bibliokeia.com/authorities/Topic/bkau-1' type='hasReciprocalAuthority'


In [63]:

for i in doc.get('hasReciprocalAuthority'):
    # request = {'authority': uri, 'uri': i['uri'], 'type': 'hasReciprocalAuthority' }
    print(i['uri'])

['https://bibliokeia.com/authorities/Topic/bkau-2']


In [15]:
def MakeLabel(elementList):
    labels = [i.elementValue.value for i in elementList]
    label = ", ".join(labels)
    
    return label

def MakeDocSubject(request, id):
    doc = {
            'id': id,
            'type': request.type,
            "creationDate": request.adminMetadata.creationDate.strftime('%Y-%m-%d'), 
            "label": f'{MakeLabel(request.elementList)}' ,
            "isMemberOfMADSCollection": request.isMemberOfMADSCollection
        }
    if request.hasVariant:
        variants = list()
        for i in request.hasVariant:
            label = [j.elementValue.value for j in i.elementList]
            label = "--".join(label)
            variants.append(label)
        doc['variant'] = variants
    for k, v in request:
        if v and k not in ['type', 'adminMetadata', 'elementList', 'hasVariant', 'note', 'isMemberOfMADSCollection']:
                uris = list()
                for i in v:
                        uri = {
                                'id': f"{id}/{k}#{i.value.split('/')[-1]}",
                                'uri': i.value, 
                                'label': i.label.value, 
                                'lang': i.label.lang,
                                'base': i.base
                                }
                        uris.append(uri)
                doc[f'{k}'] = uris
    return doc

In [13]:
solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)

doc = {
 "id":"bkau-1",
 "label":{ "remove": ["TESTE 2" ]}
}

responseSolr = solr.add([doc], commit=True)

In [21]:
solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)

In [25]:
r = solr.search(q='id:bkau-1', **{'fl': '*,[child]'})
r

In [29]:
[doc] = r.docs

In [45]:
nMeta = ["id",
"type",
"creationDate",
"label",
"isMemberOfMADSCollection",
"note",
"variant",
"_version_"]
ids = list()
for k, v in doc.items():
    if k not in nMeta:
        if type(v) == list:
            for i in v:
                ids.append(i['id'])
        else:
            ids.append(v['id'])
print(ids)

['bkau-1/hasBroaderExternalAuthority#sh85100821', 'bkau-1/hasNarrowerExternalAuthority#sh2008006144', 'bkau-1/hasNarrowerExternalAuthority#sh94007102', 'bkau-1/hasNarrowerExternalAuthority#sh85077232', 'bkau-1/hasNarrowerExternalAuthority#sh2022003400', 'bkau-1/hasNarrowerExternalAuthority#sh85104470', 'bkau-1/hasNarrowerExternalAuthority#sh2021004046', 'bkau-1/hasNarrowerExternalAuthority#sh87006717', 'bkau-1/hasNarrowerExternalAuthority#sh85070785', 'bkau-1/hasNarrowerExternalAuthority#sh95007221', 'bkau-1/hasNarrowerExternalAuthority#sh86001241', 'bkau-1/hasNarrowerExternalAuthority#sh85061789', 'bkau-1/hasNarrowerExternalAuthority#sh85039316', 'bkau-1/hasNarrowerExternalAuthority#sh86005590', 'bkau-1/hasNarrowerExternalAuthority#sh85047747', 'bkau-1/hasNarrowerExternalAuthority#sh2010003866', 'bkau-1/hasNarrowerExternalAuthority#sh85112573', 'bkau-1/hasNarrowerExternalAuthority#sh85120397', 'bkau-1/hasNarrowerExternalAuthority#sh85088886', 'bkau-1/hasNarrowerExternalAuthority#sh200

In [46]:
r = solr.delete(id=ids)
r

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">76</int>\n</lst>\n</response>\n'